In [42]:
def tweet_tag_pairs(date,tag_list):
    tag_list = sorted(tag_list)
    combinations = []
    for i in range(len(tag_list)):
        for j in range(i,len(tag_list)):
            tag1 = tag_list[i]
            tag2 = tag_list[j]
            if tag1 != tag2:
                combinations.append((date,tag1,tag2))
    return combinations
def check_duplicates(df):
    print df
    t = pd.DataFrame(columns =['f','l','tag1','tag2'])
    t['f'] = df.apply(lambda x: max(x.tag1,x.tag2),axis=1)
    t['l'] = df.apply(lambda x: min(x.tag1,x.tag2),axis=1)
    t['tag1'] = df.tag1
    t['tag2'] = df.tag2
    t['date'] = df.date
    t = t.groupby(by=["f","l"], as_index=False).last()[['date','tag1','tag2']]
    return t

## Open file

In [44]:
tweet_tag_pairs(pd.to_datetime(1),["z","a","x"])

[(Timestamp('1970-01-01 00:00:00.000000001'), 'a', 'x'),
 (Timestamp('1970-01-01 00:00:00.000000001'), 'a', 'z'),
 (Timestamp('1970-01-01 00:00:00.000000001'), 'x', 'z')]

In [39]:
check_duplicates(df)

                           date tag1 tag2
0 1970-01-01 00:00:00.000000001  wer  wah
1 1970-01-01 00:00:00.000000030  wah  wer
2 1970-01-01 00:00:00.000000001  wah  wer
3 1970-01-01 00:00:00.000000030  sdf  sfd
4 1970-01-01 00:00:00.000000030  sdf  swf


,date,tag1,tag2
0,1970-01-01 00:00:00.000000030,sdf,sfd
1,1970-01-01 00:00:00.000000030,sdf,swf
2,1970-01-01 00:00:00.000000001,wah,wer


In [32]:
x = [(pd.to_datetime(1), 'wer','wah'), (pd.to_datetime(30), 'wah','wer'), (pd.to_datetime(1), 'wah','wer'),
     (pd.to_datetime(30), 'sdf','sfd'), (pd.to_datetime(30), 'sdf','swf')]

df = pd.DataFrame(x)
df.columns = ['date','tag1','tag2']

In [24]:
df

,0,1,2
0,1970-01-01 00:00:00.000000001,wer,wah
1,1970-01-01 00:00:00.000000030,wah,wer
2,1970-01-01 00:00:00.000000030,sdf,sfd
3,1970-01-01 00:00:00.000000030,sdf,swf


In [21]:
t = pd.DataFrame(columns =['f','l'])
t['f'] = df.apply(lambda x: max(x[1],x[2]),axis=1)
t['l'] = df.apply(lambda x: min(x[1],x[2]),axis=1)

t
df['tag1'] = t['f']
df['tag2'] = t['l']


df.groupby(by=['tag1','tag2'], as_index=False).last()
df[[0,1,2]]

def check_duplicates(df):
    t = pd.DataFrame(columns =['f','l','tag1','tag2'])
    t['f'] = df.apply(lambda x: max(x.tag1,x.tag2),axis=1)
    t['l'] = df.apply(lambda x: min(x.tag1,x.tag2),axis=1)
    t['tag1'] = df.tag1
    t['tag2'] = df.tag2
    return t

,0,1,2
0,1970-01-01 00:00:00.000000001,wer,wah
1,1970-01-01 00:00:00.000000030,wah,wer
2,1970-01-01 00:00:00.000000030,sdf,sfd
3,1970-01-01 00:00:00.000000030,sdf,swf


In [66]:
max(u'hell',u'heavan')

u'hell'

In [21]:
max(pd.to_datetime(1),pd.to_datetime(0))

Timestamp('1970-01-01 00:00:00.000000001')

In [1]:
import json
import os
import pprint
import pandas as pd
import datetime

In [ ]:
## write tests ?? unit tests
is the code being run in under a minute per minute of sequence?
write out an error log?  (try except everything?!)
can text be written and read symultaneously?


In [79]:
tweets_file = '../tweet_input/tweets.txt'
f = open(tweets_file, 'r') 

# df.loc[df.cherry == 'bad', ['apple', 'banana']] = np.nan
# df[['b1', 'b2']] = df[['b1', 'b2']].where(df[['b1', 'b2']] != '', df[['a1', 'a2']].values)

output = open('../tweet_output/output.txt', 'w')
max_date = pd.to_datetime(0)
df_all = pd.DataFrame(columns =["date","tag1","tag2"])
for i, line in enumerate(f):
    # load JSON
    d = json.loads(line)
    # retrieve date and hashtag list
    if d.get(u'limit') == None:
        date = pd.to_datetime(d['created_at'])
        max_date = max(date,max_date)
        if len(d['entities']['hashtags']) > 1:
            # retrieve hashtags
            htags = [tag['text'] for tag in d['entities']['hashtags']]
            
            # build tag pairs from tweet
            pair_list = tweet_tag_pairs(date,htags)

            # dataframe of tweet
            df = pd.DataFrame(pair_list,columns =["date","tag1","tag2"])
            df_all = df_all.append(df)
            
            # Select all tweets within 60 seconds of max_time
            rolling60 = df_all[(max_date - df_all.date) < datetime.timedelta(minutes=1)]
            
            # take into account duplicates where pair1 = pair2 in another one (switched)
            rolling60 = check_duplicates(rolling60)
            
            # take into account duplicate pairs at different times
#             rolling60 = rolling60.groupby(by=["tag1","tag2"], as_index=False).last()
            
            # Calculate # unique tags
            unique_tags = len(set(list(rolling60.tag1)+list(rolling60.tag2)))
            
            # Calculate the connectivity value: (total edges * 2) / (unique tags)
            val = rolling60.shape[0]*2 / float(unique_tags)
#             print max_date, '{0:.2f}'.format(val)
            # write the value to file
            x = '{0:.2f}'.format(val)
            output.write(x+'\n')

        else:
            # even if there is zero or no hashtag in this tweet, 
            # we still want to record the running total, 
            # which is just the same as the previous one
            val = rolling60.shape[0]*2 / float(unique_tags)
            x = '{0:.2f}'.format(val)
            output.write(x+'\n')
    
df_all
# {u'limit': {u'track': 1, u'timestamp_ms': u'1459207521864'}}
# Create Tag Adjacency List
# df = pd.concat(df_list)
# a
# df.date - datetime.timedelta(minutes=1)
# df

,date,tag1,tag2
0,2016-03-28 23:23:12,hiring,PaloAlto
1,2016-03-28 23:23:12,hiring,Healthcare
2,2016-03-28 23:23:12,hiring,Job
3,2016-03-28 23:23:12,hiring,Jobs
4,2016-03-28 23:23:12,PaloAlto,Healthcare
5,2016-03-28 23:23:12,PaloAlto,Job
6,2016-03-28 23:23:12,PaloAlto,Jobs
7,2016-03-28 23:23:12,Healthcare,Job
8,2016-03-28 23:23:12,Healthcare,Jobs
9,2016-03-28 23:23:12,Job,Jobs


In [47]:
import datetime
df_all[(max_date - df_all.date) < datetime.timedelta(minutes=1)]

,date,tag1,tag2
0,2016-03-28 23:23:12,hiring,PaloAlto
1,2016-03-28 23:23:12,hiring,Healthcare
2,2016-03-28 23:23:12,hiring,Job
3,2016-03-28 23:23:12,hiring,Jobs
4,2016-03-28 23:23:12,PaloAlto,Healthcare
5,2016-03-28 23:23:12,PaloAlto,Job
6,2016-03-28 23:23:12,PaloAlto,Jobs
7,2016-03-28 23:23:12,Healthcare,Job
8,2016-03-28 23:23:12,Healthcare,Jobs
9,2016-03-28 23:23:12,Job,Jobs


In [124]:
x = pd.DataFrame([(1,3),(2,3),(4,4)])
x.append([(1,2),(3,4)])

,0,1
0,1,3
1,2,3
2,4,4
0,1,2
1,3,4


In [50]:
f = open('../tweet_output/output.txt', 'w')
res = [1.00,2.33]
for val in res:
    x = '{0:.2f}'.format(val)
    f.write(x+'\n')
    
f.close()

In [48]:
a = 1.000
str(round(a, 2))
x = '{0:.2f}'.format(1.00)
type(x)

str

In [52]:
df_all.shape[0]*2 / 

130

In [53]:
df_all

,date,tag1,tag2
0,2016-03-28 23:23:12,hiring,PaloAlto
1,2016-03-28 23:23:12,hiring,Healthcare
2,2016-03-28 23:23:12,hiring,Job
3,2016-03-28 23:23:12,hiring,Jobs
4,2016-03-28 23:23:12,PaloAlto,Healthcare
5,2016-03-28 23:23:12,PaloAlto,Job
6,2016-03-28 23:23:12,PaloAlto,Jobs
7,2016-03-28 23:23:12,Healthcare,Job
8,2016-03-28 23:23:12,Healthcare,Jobs
9,2016-03-28 23:23:12,Job,Jobs
